# **Kelompok 19**
- Claudia Mei Serin Sitio (1301190424)
- Vena Erla Candrika (1301194040)
- Clara Gracilyn Oktavia (1301184474)

In [ ]:
#import modules

import pandas as pd
import numpy as np
from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

In [ ]:
#import data

conflict_df=pd.read_csv('https://raw.githubusercontent.com/serinsitio/VISDATConflictEventsBokeh/main/conflict_data_idn.csv')
conflict_df.head()

,id,relid,year,start_year,end_year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
0,NaN,NaN,#date+year,#date+year+start,#date+year+end,NaN,NaN,NaN,NaN,NaN,...,#date+end,NaN,NaN,NaN,NaN,#affected+killed,NaN,NaN,NaN,NaN
1,141869.0,INS-1989-1-498-999,1989,1989,1989,0.0,Clear,1.0,291.0,291.0,...,1989-08-01 00:00:00.000,0.0,1.0,0.0,0.0,1,1.0,1.0,850.0,NaN
2,141872.0,INS-1991-1-498-999,1991,1991,1991,0.0,Clear,1.0,291.0,291.0,...,1991-10-27 00:00:00.000,0.0,3.0,0.0,0.0,3,3.0,3.0,850.0,NaN
3,141874.0,INS-1995-1-498-999,1995,1995,1995,0.0,Clear,1.0,291.0,291.0,...,1995-05-30 00:00:00.000,0.0,0.0,0.0,0.0,0,10.0,0.0,850.0,NaN
4,141877.0,INS-1995-1-498-2,1995,1995,1995,0.0,Clear,1.0,291.0,291.0,...,1995-11-14 00:00:00.000,1.0,0.0,0.0,0.0,1,1.0,1.0,850.0,NaN


In [ ]:
# clean the data
conflict_df=conflict_df.loc[conflict_df['year'] == '2004']
conflict_df['latitude']=conflict_df['latitude'].astype('float')
conflict_df['longitude']=conflict_df['longitude'].astype('float')
conflict_df=conflict_df.reset_index()
conflict_df=conflict_df.drop('index',axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Bokeh maps are in mercator. Convert lat lon fields to mercator units for plotting

def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df=wgs84_to_web_mercator(conflict_df,'longitude','latitude')

In [ ]:
#Establishing a zoom scale for the map. The scale variable will also determine proportions for hexbins and bubble maps so that everything looks visually appealing.

scale=2000
x=df['x']
y=df['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.

x_min=int(x.mean() - (scale * 350))
x_max=int(x.mean() + (scale * 350))
y_min=int(y.mean() - (scale * 350))
y_max=int(y.mean() + (scale * 350))

#Defining the map tiles to use. I use OSM, but you can also use ESRI images or google street maps.

tile_provider=get_provider(OSM)

#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='2004 Indonesia Conflict Events',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator'
    )

plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False

output_notebook()

In [ ]:
#function takes scale (defined above), the initialized plot object, and the converted dataframe with mercator coordinates to create a hexbin map

def hex_map(plot,df, scale,leg_label='Hexbin Heatmap'):
  r,bins=plot.hexbin(x,y,size=scale*10,hover_color='pink',hover_alpha=0.8,legend_label=leg_label)
  hex_hover = HoverTool(tooltips=[('count','@c')],mode='mouse',point_policy='follow_mouse',renderers=[r])
  hex_hover.renderers.append(r)
  plot.tools.append(hex_hover)

  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

In [ ]:
#function takes a column to determine radius and the dataframe with converted mercator coordinates to create a bubble map.
def bubble_map(plot,df,radius_col,lon,lat,conf,scale,color='orange',leg_label='Bubble Map'):
  radius=[]
  for i in df[radius_col]:
    radius.append(i*scale)

  df['radius']=radius

  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius='radius',legend_label=leg_label,hover_color='red')

  tip_label='@'+radius_col
  lat_label='@'+lat
  lon_label='@'+lon
  conf_label='@'+conf

  circle_hover = HoverTool(tooltips=[(radius_col,tip_label),('Lat:',lat_label),('Lon:',lon_label),('ConflictName:',conf_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

In [ ]:
#Create the hexbin map
hex_map(plot=plot,
               df=conflict_df,
               scale=scale,
               leg_label='Indonesia Conflict Events by Number of Events')

#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.
bubble_map(plot=plot,
           df=conflict_df,
           radius_col='high',
           leg_label='Indonesia Conflict Events by High',
           lon='longitude',
           lat='latitude',
           conf='conflict_name',
           scale=scale)

show(plot)

Link Presentasi: